# STL evaluation and performance metrics evaluation

This notebook allows you to run the `PerfObserver` on several examples and produce plots.
For each selected configuration, observers are run for all axes (p, q, r) against a single episode,
computed performance metrics are reported, and a plot is produced for each metric of interest (stable plateau detection, stepsize calculation, overshoot value, offset property, rising time property, etc.).

The process we use to produce the synthetic data presented in the paper is depicted in the figure below.

The raw traces are stored in the `data/testing` folder.

This phase of the process is storage and compute-intensive so in this
notebook we only sample a number of configurations, then compute and
display the performance metrics. It demonstrates that we have indeed
the means to compute synthetic performance metrics mentioned in the
paper.

<img src="data/HSCC2021-RE.png">

In [ ]:
import matplotlib.pyplot as plt

The parent `python_scripts` folder contains:
- `pwcsignalnumba.py`: A class representing a piecewise
  constant signal, used as the basic representation of sampled signals
  from simulations for STL evaluation.
- `props.py`: STL property observer code, generated from
  STL specifications using our in-house STL code generator.
- `perf_observer.py`: A wrapper class that uses the
  props.py observer and gathers statistics and provide plotting
  functionality.

In [ ]:
from perf_observer import load_signals, PerfObserver 

In [ ]:
params = PerfObserver.default_prop_params()
tlimit = None
episodes = ['PID/better,main/test_nominal/log/checkpoint_1/episode_0',
'PID/better,main/test_saturation/log/checkpoint_30/episode_0',
'PID/better,main/test_windgust/log/checkpoint_30/episode_0',
'ALGO/ddpg/LAYER/16x16/OBSERVED/ep,eq,er/N3000000/log/checkpoint_30/episode_0',
'ALGO/ddpg/LAYER/16x16/OBSERVED/ep,eq,er,saturation/N3000000/log/checkpoint_30/episode_0',
'ALGO/ddpg/LAYER/16x16/OBSERVED/ep,eq,er,wgx,wgy,wgz,wga/N3000000/log/checkpoint_30/episode_0',
'ALGO/ddpg/LAYER/16x16x16/OBSERVED/ep,eq,er/N3000000/log/checkpoint_30/episode_0',
'ALGO/ddpg/LAYER/16x16x16x16/OBSERVED/ep,eq,er/N3000000/log/checkpoint_30/episode_0',
'ALGO/sac/LAYER/16x16/OBSERVED/ep,eq,er/N3000000/log/checkpoint_30/episode_0',
'ALGO/sac/LAYER/16x16/OBSERVED/ep,eq,er,saturation/N3000000/log/checkpoint_30/episode_0',
'ALGO/sac/LAYER/16x16/OBSERVED/ep,eq,er,wgx,wgy,wgz,wga/N3000000/log/checkpoint_30/episode_0',
'ALGO/sac/LAYER/16x16x16/OBSERVED/ep,eq,er/N3000000/log/checkpoint_30/episode_0',
'ALGO/sac/LAYER/16x16x16x16/OBSERVED/ep,eq,er/N3000000/log/checkpoint_30/episode_0',
'ALGO/td3/LAYER/16x16/OBSERVED/ep,eq,er/N3000000/log/checkpoint_30/episode_0',
'ALGO/td3/LAYER/16x16x16/OBSERVED/ep,eq,er/N3000000/log/checkpoint_30/episode_0',
'ALGO/td3/LAYER/16x16x16x16/OBSERVED/ep,eq,er/N3000000/log/checkpoint_30/episode_0',
'NOMINAL/nn,n/ALGO/ddpg/LAYER/16x16/OBSERVED/ep,eq,er,saturation/N3000000/log/checkpoint_30/episode_0',
'NOMINAL/nn,n/ALGO/ddpg/LAYER/16x16/OBSERVED/ep,eq,er,wgx,wgy,wgz,wga/N3000000/log/checkpoint_30/episode_0',
'NOMINAL/nn,n/ALGO/ddpg/LAYER/32x32/OBSERVED/ep,eq,er,saturation/N3000000/log/checkpoint_30/episode_0',
'NOMINAL/nn,n/ALGO/ddpg/LAYER/32x32/OBSERVED/ep,eq,er,wgx,wgy,wgz,wga/N3000000/log/checkpoint_30/episode_0',
'NOMINAL/nn,n/ALGO/sac/LAYER/16x16/OBSERVED/ep,eq,er,saturation/N3000000/log/checkpoint_30/episode_0',
'NOMINAL/nn,n/ALGO/sac/LAYER/16x16/OBSERVED/ep,eq,er,wgx,wgy,wgz,wga/N3000000/log/checkpoint_30/episode_0',
'NOMINAL/nn,n/ALGO/sac/LAYER/32x32/OBSERVED/ep,eq,er,saturation/N3000000/log/checkpoint_30/episode_0',
'NOMINAL/nn,n/ALGO/sac/LAYER/32x32/OBSERVED/ep,eq,er,wgx,wgy,wgz,wga/N3000000/log/checkpoint_30/episode_0',
'NOMINAL/n,nn/ALGO/sac/LAYER/16x16/OBSERVED/ep,eq,er/TESTS/saturation/log/checkpoint_30/episode_0',
'NOMINAL/n,nn/ALGO/sac/LAYER/16x16/OBSERVED/ep,eq,er/TESTS/windgust/log/checkpoint_30/episode_0',
'NOMINAL/n,nn/ALGO/sac/LAYER/32x32/OBSERVED/ep,eq,er/TESTS/saturation/log/checkpoint_30/episode_0',
'NOMINAL/n,nn/ALGO/sac/LAYER/32x32/OBSERVED/ep,eq,er/TESTS/windgust/log/checkpoint_30/episode_0',
'NOMINAL/n,nn/ALGO/sac/LAYER/64x64/OBSERVED/ep,eq,er/TESTS/saturation/log/checkpoint_30/episode_0',
'NOMINAL/n,nn/ALGO/sac/LAYER/64x64/OBSERVED/ep,eq,er/TESTS/windgust/log/checkpoint_30/episode_0']


The signal computed by the property observer at each step is in
magental, signal in blue and query in green.

In [ ]:
po = PerfObserver(10000, params)
qo = PerfObserver(10000, params)
ro = PerfObserver(10000, params)

for file in episodes:
    po.reset_stats()
    qo.reset_stats()
    ro.reset_stats()
    signals = load_signals('data/testing/' + file + '.', t_is_hex=True, tlimit=tlimit)
    # instanciate
    t = signals['t']
    r = signals['r']
    qr = signals['query_r']
    p = signals['p']
    qp = signals['query_p']
    q = signals['q']
    qq = signals['query_q']
    ro.observe(t, r, qr, save_eval = True)
    po.observe(t, p, qp, save_eval = True)
    qo.observe(t, q, qq, save_eval = True)
    print("------------------")
    print(file)
    print("----- p axis-----")
    print(po.get_stats_dict())
    po.all_plots()
    print("----- q axis-----")
    print(qo.get_stats_dict())
    qo.all_plots()
    print("----- r axis-----")
    print(ro.get_stats_dict())
    ro.all_plots()
    print("------------------")
      

